In [1]:
import NI
import NN
import h5py
import json
import time
import pyccl
import scipy
import numpy
from itertools import product
from matplotlib import pyplot
from matplotlib.gridspec import GridSpec

In [2]:
#Data path

TYPE = 'NI'
PATH = '/pscratch/sd/y/yhzhang/TensorCloud/'

DATA_PATH = PATH + 'DATA/'
PLOT_PATH = PATH + 'PLOT/' + TYPE + '/'

In [3]:
#Load in Data

BIN_SIZE = 4
GRID_SIZE = 128
DATA_SIZE = 100000

with h5py.File(DATA_PATH + 'DATA.hdf5','r') as DATA:
    
    REDSHIFT_GRID = numpy.array(DATA['realizations/z'][:GRID_SIZE], dtype = 'float32')
    DATA_GRID = numpy.array(DATA['realizations/pdfs'][:DATA_SIZE, :BIN_SIZE, :GRID_SIZE], dtype = 'float32')

In [4]:
# COSMOLOGY

with open(DATA_PATH + 'COSMO.json', 'r') as cosmo:
    
    COSMO = json.load(cosmo)

COSMO_CCL = pyccl.Cosmology(
    h = COSMO['H'],
    w0 = COSMO['W0'],
    wa = COSMO['WA'], 
    n_s = COSMO['NS'], 
    A_s = COSMO['AS'],
    m_nu = COSMO['MNU'],  
    Neff = COSMO['NEFF'],
    T_CMB = COSMO['TCMB'], 
    Omega_k = COSMO['OMEGAK'], 
    Omega_c = COSMO['OMEGAC'], 
    Omega_b = COSMO['OMEGAB'], 
    matter_power_spectrum = 'halofit',  
    transfer_function = 'boltzmann_camb', 
    extra_parameters = {'camb': {'kmax': 100, 'lmax': 100, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [5]:
#Restrcture the data

PSI_GRID = numpy.mean(DATA_GRID, axis = 0)
SIGMA_GRID = numpy.std(DATA_GRID, axis = 0)
A_GRID = numpy.array(1 / (1 + REDSHIFT_GRID), dtype = 'float32')

CHI_GRID = COSMO_CCL.comoving_radial_distance(a = A_GRID)
PHI_GRID = PSI_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + REDSHIFT_GRID)) * COSMO['H'] * 100000 / scipy.constants.c

ELL_SIZE = 200
ELL_DATA = numpy.logspace(2, 4, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_DATA)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out = numpy.zeros((ELL_SIZE + 1, GRID_SIZE)) + numpy.inf, where = CHI_MESH > 0))

POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE))
for GRID_INDEX in range(GRID_SIZE):

    POWER_GRID[:,GRID_INDEX] = COSMO_CCL.linear_matter_power(k = SCALE_GRID[:,GRID_INDEX], a = A_GRID[GRID_INDEX])

GROWTH_RATE = pyccl.background.growth_rate(cosmo=COSMO_CCL,a = A_GRID)

In [6]:
#Define angular power spectrum

def c_m1_m2(amplitude, phi_m1_grid, phi_m2_grid, chi_grid, power_grid, growth_grid, type):

    pni = scipy.interpolate.interp1d(chi_grid, power_grid, kind = type)
    
    phi_m1 = scipy.interpolate.interp1d(chi_grid, phi_m1_grid, kind = type)
    
    phi_m2 = scipy.interpolate.interp1d(chi_grid, phi_m2_grid/growth_grid, kind = type)
    
    function = lambda chi: phi_m1(chi) * phi_m2(chi) * pni(chi) / (chi**2)
    
    c_m1_m2_ell = amplitude * scipy.integrate.fixed_quad(func = function, a = chi_grid.min(), b = chi_grid.max(), n = 500)[0]

    return c_m1_m2_ell

In [17]:
#Compare different numeric equations
AMPLITUDE = (0.0134)*(COSMO['OMEGAM'] + COSMO['OMEGAN'])*numpy.sqrt( (1 + 3 / (2 * ELL_DATA + 1)) * (1 + 1 / (2 * ELL_DATA + 1)) * (1 - 1 / (2 * ELL_DATA + 1)) * (1 - 3 / (2 * ELL_DATA + 1)))
C_DATA = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
C_DATA1 = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
C_DATA2 = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
C_DATA3 = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))
C_CCL = numpy.zeros((BIN_SIZE, BIN_SIZE, ELL_SIZE + 1))

T0 = time.time()
C_DATA = NI.function(amplitude = AMPLITUDE, phi_grid = PHI_GRID, chi_grid = CHI_GRID, power_grid = POWER_GRID,growth_grid=GROWTH_RATE)

T1 = time.time()
print(T1 - T0)
IAb=numpy.ones(len(REDSHIFT_GRID))
for i in range(len(REDSHIFT_GRID)):
    IAb[i] = 0.0134 * (COSMO['OMEGAM'] + COSMO['OMEGAN']) /GROWTH_RATE[i]
IA_bias=[REDSHIFT_GRID, IAb]
POWER_test_GRID=numpy.zeros((ELL_SIZE + 1, GRID_SIZE))
for i in range(ELL_SIZE + 1):
    for j in range(GRID_SIZE):
        POWER_test_GRID[i][j] =POWER_GRID[i][j]*(IAb[j])
C_DATA_test = NN.function(amplitude = 1, phi_grid = PHI_GRID, chi_grid = CHI_GRID, power_grid = POWER_test_GRID, redshift_grid = REDSHIFT_GRID)
print(C_DATA_test[0][0]/C_DATA[0][0])

2.417517900466919
[1.00001875 1.00001472 1.00001097 1.00000719 1.0000031  0.99999855
 0.99999348 0.99998797 0.99998216 0.99997629 0.99997059 0.99996532
 0.99996068 0.99995681 0.99995374 0.99995134 0.99994941 0.99994764
 0.9999457  0.99994331 0.99994037 0.99993694 0.99993331 0.9999298
 0.99992674 0.99992426 0.99992237 0.99992088 0.99991958 0.99991822
 0.9999167  0.99991494 0.99991298 0.99991088 0.99990881 0.99990696
 0.99990541 0.9999042  0.99990318 0.99990219 0.99990107 0.99989977
 0.99989835 0.99989696 0.99989574 0.99989473 0.99989387 0.99989301
 0.99989209 0.99989109 0.99989011 0.9998892  0.99988838 0.99988761
 0.99988686 0.9998861  0.99988535 0.99988463 0.99988395 0.99988329
 0.99988264 0.99988202 0.99988142 0.99988083 0.99988025 0.99987968
 0.99987914 0.99987861 0.99987809 0.99987759 0.99987711 0.99987665
 0.9998762  0.99987577 0.99987534 0.99987493 0.99987452 0.99987413
 0.99987374 0.99987336 0.99987299 0.99987263 0.99987228 0.99987194
 0.99987161 0.99987128 0.99987096 0.99987065 